In [ ]:
from datetime import *
from dateutil.parser import parse
import numpy as np
import pandas as pd

# Make Sure paths match up


data = pd.read_csv('/content/drive/MyDrive/ai-fall20/datasets/predict-future-sales/sales_train.csv')
tree = pd.read_csv('/content/drive/MyDrive/ai-fall20/datasets/predict-future-sales/items.csv')
test = pd.read_csv('/content/drive/MyDrive/ai-fall20/datasets/predict-future-sales/test.csv')



# Manipulate Train Data
from dateutil.parser import parse
data.date = data.date.astype('str')
data.date = data.date.apply(lambda x: parse(x))
data.date = data.date.apply(lambda x: pd.to_datetime(x,
    format='%Y-%m-%dT',
    errors='coerce'))
data['month'] = data.date.dt.month
data['year'] = data.date.dt.year
data = data.groupby(['item_id','shop_id','date_block_num'], as_index=False).agg({'item_price':'sum', "item_cnt_day":'sum','month':'first', 'year':'first'})


# summed counts by month, change name 
data = data.rename(columns={'item_cnt_day':'monthly_sales'})
data = data.reset_index()
data = data.drop('index', axis=1)
tree = tree.drop('item_name', axis=1)

# extra features
fin_map = {2013:[63.6,1.8,6.5,519.6], 2014:[64.1, 0.7, 11.4, 552.7], 2015:[62.4,-2.5, 12.9, 575.3]}

# Gist - SAVE!!! for now the only way I know how to create a dictionary with two columns
# to map to a dataframe
branch = tree.set_index('item_id').T.to_dict('records')[0]
data['categories'] = data['item_id'].map(branch)
data['finance'] = data['year'].map(fin_map)


# Manipulate Test Data
test = test.assign(date_block_num = 34)
test  = test.assign(month = 11)
test['categories'] = test['item_id'].map(branch)
test = test.assign(finance)


#last things to share csv, dropping and rearranging
col_rearrange = ['shop_id','item_id','date_block_num','month','categories','monthly_sales']
data = data[col_rearrange]





#WriteOut to csv
data.to_csv("/content/drive/MyDrive/ai-fall20/datasets/predict-future-sales/Train_mod.csv", index=False)
test.to_csv("/content/drive/MyDrive/ai-fall20/datasets/predict-future-sales/Test_mod.csv", index=False)
data.to_parquet("/content/drive/MyDrive/ai-fall20/datasets/predict-future-sales/train_mod.parquet", engine="pyarrow")
test.to_parquet("/content/drive/MyDrive/ai-fall20/datasets/predict-future-sales/test_mod.parquet", engine="pyarrow")